In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [327]:
# Imports
import numpy as np
import pandas as pd
from tqdm import tqdm 
from sklearn.model_selection import train_test_split

In [4]:
img_feat_path = "../data/features_densenet.npy"
text_feat_path = "../data/features_bert_1000.npy"

In [5]:

img_features = np.load(img_feat_path)

In [6]:

text_features = np.load(text_feat_path)


In [7]:
text_features[0]

array([ 5.29073298e-01, -1.71541721e-01,  3.08802336e-01,  7.86855537e-03,
        3.14220130e-01, -2.25084871e-01, -4.98234421e-01,  2.22875521e-01,
       -7.08729446e-01, -3.04852515e-01,  3.68502110e-01, -6.68642461e-01,
       -2.49022081e-01,  6.13008976e-01,  1.49190743e-02,  5.75145006e-01,
       -7.28979111e-02,  2.54051745e-01,  1.46331578e-01,  1.17061064e-01,
        2.69857317e-01, -2.34132439e-01, -5.62548995e-01, -1.31890620e-03,
       -1.61656335e-01, -2.22178608e-01,  1.16699003e-01, -6.48799613e-02,
        1.89696312e-01, -5.23374453e-02,  1.48475945e-01, -1.26938090e-01,
       -7.96284601e-02, -3.05142701e-01, -3.52363884e-01, -5.94661772e-01,
       -1.37687698e-01, -1.38169765e-01, -2.83382952e-01,  6.88940380e-03,
       -7.41415024e-01,  1.05075583e-01,  2.42597133e-01,  1.09096572e-01,
       -5.24445832e-01,  4.32214960e-02, -5.83388880e-02,  8.40286240e-02,
       -9.80826989e-02,  3.30672935e-02, -3.73284042e-01,  1.47073761e-01,
       -3.63114566e-01,  

In [9]:
text_features.shape

(368900, 768)

In [10]:
img_features.shape

(82783, 7, 7, 1024)

In [11]:
img_features = img_features.reshape(82783,7*7*1024)

In [12]:
img_features.shape

(82783, 50176)

In [13]:
df = pd.read_csv("../data/train_data_1000.csv")
df.head()

,image_id,ques_id,question,answer
0,458752,458752000,What is this photo taken looking through?,net
1,458752,458752001,What position is this man playing?,pitcher
2,458752,458752002,What color is the players shirt?,orange
3,458752,458752003,Is this man a professional baseball player?,yes
4,262146,262146000,What color is the snow?,white


In [14]:
# Get these values for mapping

answers = []
img_id = []
ques_id = []
for _,row in tqdm(df.iterrows()):
    answers.append(str(row.answer))
    img_id.append(row.image_id)
    ques_id.append(row.ques_id)

368900it [00:37, 9781.10it/s] 


In [15]:
orig_img_id = sorted(np.unique(img_id))

## Write a custom generator to fetch image and text features

In [16]:
batch = np.random.choice(a = ques_id, 
                                     size = 64)

In [17]:
batch

array([332459000, 130741010,  45885005, 191098001, 313721000, 198557000,
       127994000, 223836002, 319031000,  37955013, 374725002, 256040000,
        96845019,  42804001, 390159002, 249663000, 156700003, 376241052,
       433285000, 554360001, 156284003, 336426002, 312334002,  32339002,
       175341000, 478967001, 129722001, 342325001, 340653001, 259398000,
        32804007, 358411000, 371814000, 540457000, 237186002, 168873005,
        55536009, 132699000, 451420001,  33094003, 568461005,  46230010,
       311377002, 213135009, 448352001, 289852004,  50648002, 499061000,
       440520002,  73721001,  61842042, 228155002, 329640027, 397212008,
       194294000, 139309001, 551867002, 579539011, 122352010,  20279002,
       486227000, 179037006, 258337001,  27675001])

In [163]:
X_train =[]
y_train = [] 
for c,val in enumerate(tqdm(ques_id[:100])):

    get_text_index = ques_id.index(val)
    get_text_index = ques_id.index(val)
    get_img_index = orig_img_id.index(img_id[get_text_index])
    txt_features = text_features[get_text_index]
    txt_features = txt_features.reshape(768)
    image_features = img_features[get_img_index]
    image_features = image_features.reshape(7*7*1024)
#     print(txt_features.shape,image_features.shape)
    input = np.concatenate((image_features,txt_features))
    input=input.reshape(1,50944)
#     input = np.array(input).flatten()
#             input = np.concatenate(txt_features.ravel(), image_features.ravel())
    output = answers[get_text_index]
    if c==0:
        X_train = input
    else:
        X_train = np.vstack((X_train,input))
    y_train += [ output ]


100%|██████████| 100/100 [00:00<00:00, 113.98it/s]

In [149]:
X_train.shape

(100, 50944)

In [245]:
onehot = pd.get_dummies(answers)
target_labels = onehot.columns
target = onehot.as_matrix()

/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [247]:
target_labels = list(target_labels)
len(target_labels)

999

In [248]:
assert target_labels.index(y_train[10]) == np.argmax(target[10]), "Error in mapping"

In [356]:
def custom_generator(ques_id,answers=answers, batch_size = 1):
    
    while True:
    
          # For one-hot labeling
        onehot = pd.get_dummies(answers)
        target_labels = onehot.columns
        target = onehot.as_matrix()

        # Select files (paths/indices) for the batch
        batch = np.random.choice(a = ques_id, 
                                     size = batch_size)
        batch_x = np.array([])
        batch_y = np.array([])
#         print(batch)
        # Read in each input, perform preprocessing and get labels
        for c,val in enumerate(batch):
            
            get_text_index = ques_id.index(val)
            get_text_index = ques_id.index(val)
            get_img_index = orig_img_id.index(img_id[get_text_index])
            txt_features = text_features[get_text_index]
            txt_features = txt_features.reshape(768)
            image_features = img_features[get_img_index]
            image_features = image_features.reshape(7*7*1024)
            input = np.concatenate((image_features,txt_features))
            input=input.reshape(1,50944)
            output = target[get_text_index]
            
            if c==0:
                batch_x = input
                batch_y = output
            else:
#                 print(batch_x.shape)
                batch_x = np.vstack((batch_x,input))
                batch_y = np.vstack((batch_y,output))
            
#         print(batch_x.shape)
#         print(len(batch_y))
        yield (batch_x, batch_y)
        


## Get train and val generator

In [405]:
# Train-test split
train_ques_id, val_ques_id, train_answers, val_answers = train_test_split(ques_id, answers, test_size=0.15)

In [422]:
train_gen=custom_generator(train_ques_id,batch_size=100)
val_gen=custom_generator(val_ques_id,batch_size=100)

In [413]:
x,y=next(val_gen)

/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


KeyboardInterrupt: 

In [ ]:
print(x.shape,y.shape)

## Define model

In [423]:
from keras.models import Sequential
from keras.layers.core import Reshape, Activation, Dropout
from keras.layers import LSTM, Concatenate, Dense,InputLayer, Flatten

In [424]:
# create model
del model
model = Sequential()
model.add(InputLayer(input_shape=(50944,)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(999, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 128)               6520960   
_________________________________________________________________
dropout_15 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 512)               66048     
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 999)               512487    
Total params: 7,099,495
Trainable params: 7,099,495
Non-trainable params: 0
_________________________________________________________________


In [425]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=train_gen, steps_per_epoch= len(train_answers)//10000,epochs=100,
                    validation_data=val_gen, validation_steps=len(val_answers)//10000,
                    use_multiprocessing=True,
                    workers=6)

Epoch 1/100


/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/anaconda3/envs/keras/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a fu

31/31 [==============================] - 73s 2s/step - loss: 4.8978 - acc: 0.2068 - val_loss: 4.4069 - val_acc: 0.2500
Epoch 2/100
31/31 [==============================] - 61s 2s/step - loss: 4.3374 - acc: 0.2219 - val_loss: 4.0441 - val_acc: 0.2160
Epoch 3/100
31/31 [==============================] - 63s 2s/step - loss: 4.0601 - acc: 0.2400 - val_loss: 3.7013 - val_acc: 0.2540
Epoch 4/100
31/31 [==============================] - 63s 2s/step - loss: 4.0210 - acc: 0.2187 - val_loss: 3.5284 - val_acc: 0.2220
Epoch 5/100
31/31 [==============================] - 62s 2s/step - loss: 3.8183 - acc: 0.2342 - val_loss: 3.7528 - val_acc: 0.2180
Epoch 6/100
31/31 [==============================] - 63s 2s/step - loss: 3.8970 - acc: 0.2300 - val_loss: 4.0308 - val_acc: 0.2020
Epoch 7/100
31/31 [==============================] - 65s 2s/step - loss: 3.9322 - acc: 0.2187 - val_loss: 4.0480 - val_acc: 0.1860
Epoch 8/100
31/31 [==============================] - 61s 2s/step - loss: 3.9568 - acc: 0.2216 -

In [366]:
model.save("stage1.h5")

In [ ]:
!ls -h

In [382]:
np.argmax(model.predict(X_train[14:15]))

363

In [378]:
np.argmax(target[11])

994

In [383]:
target_labels[363]

'frisbee'

In [ ]:
from keras.preprocessing import image
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input

def get img_feat:
    